In [ ]:
############ YAHOO FINANCE API #########

In [23]:

def fetch_stock_data(symbol, interval='2m', range_span='1d'):
    """
    Fetches stock data for a given symbol from Yahoo Finance.

    :param symbol: The stock symbol (e.g., 'NAUKRI.NS')
    :param interval: The interval for the stock data (default is '2m')
    :param range_span: The date range for the stock data (default is '1d')
    :return: A dictionary with stock data.
    """
    YAHOO_FINANCE_URL = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"

    params = {
        "region": "US",
        "lang": "en-US",
        "includePrePost": "false",
        "interval": interval,
        "useYfid": "true",
        "range": range_span,
        "corsDomain": "finance.yahoo.com",
        ".tsrc": "finance"
    }

    response = requests.get(YAHOO_FINANCE_URL, params=params)
    status_code = response.status_code

    if status_code != 200:
        raise RuntimeError(f"An error occurred. HTTP Code: {status_code}.")
    else:
        return response.json()

# Example usage
stock_symbol = 'NAUKRI.NS'  # Example stock symbol

try:
    stock_data = fetch_stock_data(stock_symbol)
    # Process and print relevant information from stock_data
    print(stock_data)
except RuntimeError as e:
    print(f"Error fetching data for {stock_symbol}: {e}")



Error fetching data for NAUKRI.NS: An error occurred. HTTP Code: 403.


In [ ]:
############ ALPHA VANTAGE API #############

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
from config import ALPHA_VANTAGE_API_KEY  

def get_stock_data(symbol):
    app = TimeSeries('ALPHA_VANTAGE_API_KEY')
    data, meta_data = app.get_daily_adjusted(symbol, outputsize='compact')

    today = datetime.now().strftime('%Y-%m-%d')  # Get today's date in YYYY-MM-DD format

    if today in data:
        return {
            'symbol': symbol,
            'data': data[today]
        }
    else:
        print(f"No data available for {symbol} on {today}")
        return None

# Retrieve and store stock data for AAPL, META, and TSLA in an array
stock_prices_today = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices_today.append(stock_data)

# Displaying the array of stock prices
print(stock_prices_today)


[{'symbol': 'AAPL', 'data': {'1. open': '191.49', '2. high': '192.93', '3. low': '190.825', '4. close': '191.31', '5. adjusted close': '191.31', '6. volume': '39269515', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}, {'symbol': 'META', 'data': {'1. open': '339.21', '2. high': '342.92', '3. low': '338.58', '4. close': '341.49', '5. adjusted close': '341.49', '6. volume': '10691386', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}, {'symbol': 'TSLA', 'data': {'1. open': '242.04', '2. high': '244.01', '3. low': '231.4', '4. close': '234.21', '5. adjusted close': '234.21', '6. volume': '117754574', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}}]
